In [1]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use("Agg")
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
import torch
import time

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2023-06-05 15:38:52.324972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = pd.read_csv("cleaned_data.csv")
X = data.iloc[:,0:-1]
Y = data.iloc[:,-1]
y = np.zeros((len(Y), 10))
for i in range(len(Y)):
    y[i][int(Y[i])] = 1 
labels = ["C201", "C203", "C204", "C205", "C206", "C207", "C208", "C210", "DC", "MC"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

In [5]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model = Sequential()
model.add(Dense(51, input_shape=(51,), activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
480/480 [==============================] - 1s 802us/step - loss: 0.4995 - accuracy: 0.4979
Epoch 2/100
480/480 [==============================] - 0s 815us/step - loss: 0.1161 - accuracy: 0.8108
Epoch 3/100
480/480 [==============================] - 0s 801us/step - loss: 0.0814 - accuracy: 0.8873
Epoch 4/100
480/480 [==============================] - 0s 807us/step - loss: 0.0666 - accuracy: 0.9087
Epoch 5/100
480/480 [==============================] - 0s 792us/step - loss: 0.0558 - accuracy: 0.9292
Epoch 6/100
480/480 [==============================] - 0s 797us/step - loss: 0.0444 - accuracy: 0.9423
Epoch 7/100
480/480 [==============================] - 0s 802us/step - loss: 0.0434 - accuracy: 0.9488
Epoch 8/100
480/480 [==============================] - 0s 795us/step - loss: 0.0411 - accuracy: 0.9502
Epoch 9/100
480/480 [==============================] - 0s 804us/step - loss: 0.0363 - accuracy: 0.9519
Epoch 10/100
480/480 [==============================] - 0s 799us/step - l

480/480 [==============================] - 0s 885us/step - loss: 0.0102 - accuracy: 0.9869
Epoch 80/100
480/480 [==============================] - 0s 790us/step - loss: 0.0091 - accuracy: 0.9842
Epoch 81/100
480/480 [==============================] - 0s 794us/step - loss: 0.0099 - accuracy: 0.9823
Epoch 82/100
480/480 [==============================] - 0s 807us/step - loss: 0.0126 - accuracy: 0.9800
Epoch 83/100
480/480 [==============================] - 0s 791us/step - loss: 0.0057 - accuracy: 0.9917
Epoch 84/100
480/480 [==============================] - 0s 801us/step - loss: 0.0097 - accuracy: 0.9858
Epoch 85/100
480/480 [==============================] - 0s 796us/step - loss: 0.0073 - accuracy: 0.9873
Epoch 86/100
480/480 [==============================] - 0s 828us/step - loss: 0.0081 - accuracy: 0.9869
Epoch 87/100
480/480 [==============================] - 0s 873us/step - loss: 0.0057 - accuracy: 0.9902
Epoch 88/100
480/480 [==============================] - 0s 868us/step - loss:

In [15]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

38/38 [==============================] - 0s 842us/step - loss: 0.0057 - accuracy: 0.9908
Accuracy: 99.08


In [7]:
import pickle
with open("model_nn.sav", "wb") as file:
    pickle.dump(model, file)